<a href="https://colab.research.google.com/github/rodrigobernall/ds4a_group30/blob/master/scripts/EDA/origin_destination_matrix_creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando las librerías

In [0]:
!pip install geopandas

In [0]:
import pandas as pd
import geopandas as gpd

# Importando los datos

In [0]:
url = 'https://www.dropbox.com/s/7ixw7pe46q3theo/green_trips_nta.csv.gz?dl=1'
green_trips_nta = pd.read_csv(url, compression='gzip')
green_trips_nta = green_trips_nta.drop(columns=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])

In [27]:
url = 'https://www.dropbox.com/s/pjovxyr8w6x6y5x/zones.csv?dl=1'
zones = pd.read_csv(url)
zones

,location_id,borough,zone,service_zone,nta_code
0,1,EWR,Newark Airport,EWR,NJ01
1,2,Queens,Jamaica Bay,Boro Zone,QN61
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone,BX31
3,4,Manhattan,Alphabet City,Yellow Zone,MN22
4,5,Staten Island,Arden Heights,Boro Zone,SI48
5,6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone,SI14
6,7,Queens,Astoria,Boro Zone,QN70
7,8,Queens,Astoria Park,Boro Zone,QN70
8,9,Queens,Auburndale,Boro Zone,QN48
9,10,Queens,Baisley Park,Boro Zone,QN76


In [29]:
url = 'https://www.dropbox.com/s/8gys15vzjpd1ibu/demographics.csv?dl=1'
demographics = pd.read_csv(url)
demographics

,nta_name,borough,nta_code,population,under_5_years,5-9_years,10-14_years,15-19_years,20-24_years,25-29_years,30-34_years,35-39_years,40-44_years,45-49_years,50-54_years,55-59_years,60-64_years,over_65_years,median_age,people_per_acre,households,"less_than_10,000",10000_to_14999,15000_to_24999,25000_to_34999,35000_to_49999,50000_to_74999,75000_to_99999,100000_to_149999,150000_to_199999,200000_or_more,median_income,mean_income
0,Allerton-Pelham Gardens,Bronx,BX31,28903,1679,1706,1763,2039,1964,1703,1798,1654,1937,2203,2027,1785,1619,5026,40,39.8,9587,836,348,797,773,1160,1764,1155,1562,765,427,61638,78489
1,Annadale-Huguenot-Prince's Bay-Eltingville,Staten Island,SI01,27770,1397,1698,1817,1880,1720,1594,1487,1814,2073,2274,2246,1961,1869,3940,41,8.4,9885,467,253,571,405,1008,1523,1346,2075,1086,1151,88288,109187
2,Arden Heights,Staten Island,SI48,25238,1507,1540,1596,1752,1614,1561,1726,1828,1914,1892,1910,1945,1763,2690,39,21.8,8960,304,256,337,516,707,1421,1611,2021,1047,740,89570,101627
3,Astoria,Queens,QN70,78793,3480,3037,3060,3392,6630,11586,9252,6843,5633,4944,4599,4018,3373,8946,34,87.3,34310,2772,1782,3673,2816,4725,6463,4557,4698,1627,1197,54882,70094
4,Auburndale,Queens,QN48,19996,917,966,1063,1168,1214,1307,1296,1336,1451,1557,1659,1560,1326,3176,43,25.5,7122,270,349,445,632,690,1417,1060,1237,589,433,70772,84402
5,Baisley Park,Queens,QN76,34603,2198,2389,2569,2923,2699,2430,2264,2191,2414,2763,2585,2013,1469,3696,35,35.7,10730,637,385,820,941,1575,2186,1532,1718,629,307,59357,72664
6,Bath Beach,Brooklyn,BK27,29931,1766,1668,1672,1581,1922,2234,2308,2241,2006,2080,2097,1930,1653,4773,39,62.3,11125,897,768,1091,932,1407,2016,1176,1606,711,521,55193,76037
7,Battery Park City-Lower Manhattan,Manhattan,MN25,39699,2272,1055,691,1453,5203,7559,5978,4206,2823,1945,1589,1398,1182,2345,31,82.7,21216,1197,420,800,609,960,1909,2416,3878,2377,6650,125434,185275
8,Bay Ridge,Brooklyn,BK31,78607,4571,3998,3772,3802,4700,7214,6738,6207,5718,5618,5541,5090,4340,11298,39,50.5,34991,2654,1864,3575,3067,4056,5848,4154,4869,2498,2406,57980,82209
9,Bayside-Bayside Hills,Queens,QN46,43808,2106,2199,2353,2694,2504,2832,2935,3014,3158,3569,3531,3197,2698,7018,42,23.6,16008,578,494,1284,1073,1680,2506,2360,3047,1710,1276,79120,94806


## Computing NTA centroids

In [0]:
url = 'https://www.dropbox.com/s/pt2qx83gxzpsacp/nta.json?dl=1'
nta_polygons = gpd.read_file(url)

In [0]:
url = 'https://www.dropbox.com/s/3ts67qym904tess/nta_centroids.json?dl=1'
nta_centroids = gpd.read_file(url)

## Origin-destination matrices

In [0]:
most_common_destinations_green = green_trips_nta['dropoff_nta'].value_counts()
most_common_origins_green = green_trips_nta['pickup_nta'].value_counts()

In [0]:
ODM_green = green_trips_nta.groupby(['pickup_nta', 'dropoff_nta']).size()
ODM_green = pd.DataFrame(ODM_green)
ODM_green = ODM_green.sort_values(by=[0], ascending=False)
ODM_green.columns = ['trips']

Adding the centroids of the NTAs to the ODM (Origin-Destination Matrix)

In [61]:
a = ODM_green.reset_index()
a = a.merge(nta_centroids, left_on='pickup_nta', right_on='id')
a = a.merge(nta_centroids, left_on='dropoff_nta', right_on='id', suffixes=('_pickup', '_dropoff'))
a

,pickup_nta,dropoff_nta,trips,id_pickup,geometry_pickup,id_dropoff,geometry_dropoff
0,QN70,QN70,41981,QN70,POINT (40.76150 -73.91970),QN70,POINT (40.76150 -73.91970)
1,BK73,QN70,2499,BK73,POINT (40.71492 -73.95863),QN70,POINT (40.76150 -73.91970)
2,QN28,QN70,5973,QN28,POINT (40.75732 -73.88532),QN70,POINT (40.76150 -73.91970)
3,MN11,QN70,279,MN11,POINT (40.80434 -73.95129),QN70,POINT (40.76150 -73.91970)
4,BK37,QN70,303,BK37,POINT (40.67124 -73.98390),QN70,POINT (40.76150 -73.91970)
5,MN09,QN70,221,MN09,POINT (40.80789 -73.96254),QN70,POINT (40.76150 -73.91970)
6,MN33,QN70,455,MN33,POINT (40.79002 -73.94575),QN70,POINT (40.76150 -73.91970)
7,MN03,QN70,206,MN03,POINT (40.81827 -73.94077),QN70,POINT (40.76150 -73.91970)
8,QN29,QN70,3519,QN29,POINT (40.73950 -73.87712),QN70,POINT (40.76150 -73.91970)
9,QN17,QN70,567,QN17,POINT (40.72144 -73.84767),QN70,POINT (40.76150 -73.91970)
